In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
import xvec
import os

from dask.distributed import Client, LocalCluster
import dask

from utils import download_and_read_fwi_data_for_dates

## Constants Set Up

In [ ]:
mtbs_raster_data_dir = "./data/mtbs_rasters"
mtbs_perimeter_data_path = "./data/mtbs_perimeters/mtbs_perims_DD.shp"
eia_utility_service_areas_path = "./data/eia_service_areas/Electric_Retail_Service_Territories.geojson"
fwi_variable = "GPM.LATE.v5_FWI"
memory_per_worker = "30GB"
n_workers = 6

CRS = "EPSG:4269"

In [ ]:
cluster = LocalCluster(
    n_workers=n_workers,
    threads_per_worker=2,  
    memory_limit=memory_per_worker  
)

client = Client(cluster)

## Step 1)

First, we read in the MTBS Wildfire data files and package them in a list of dictionaries



In [ ]:

mtbs_raster_files = os.listdir(mtbs_raster_data_dir)

mtbs_gdf = gpd.read_file(mtbs_perimeter_data_path)

input_data = []

for file in mtbs_raster_files:
    data_struct = {}

    file_info = file[:-4].split("_")
    state = file_info[1]
    year = int(file_info[2])

    data_struct["state"] = state
    data_struct["year"] = year

    da_mtsb_raster = rioxarray.open_rasterio(f"{mtbs_raster_data_dir}/{file}")
    da_mtsb_raster = da_mtsb_raster.rio.reproject(CRS)
    # We filter here to include only values of meaningful fire severity
    da_mtsb_raster = da_mtsb_raster.where((da_mtsb_raster >= 2) & (da_mtsb_raster < 5))
    da_mtsb_raster.name = "fire_severity"
    data_struct["da_fire_severity"] = da_mtsb_raster

    gdf_filtered = (mtbs_gdf.loc[(mtbs_gdf["Incid_Type"]=="Wildfire") & (mtbs_gdf["Ig_Date"].dt.year == data_struct["year"]) & (mtbs_gdf["Event_ID"].str.startswith(data_struct["state"]))])
    gdf_temp = gdf_filtered.set_index("Event_ID")[["geometry"]]
    gdf_temp = gdf_temp.to_crs(da_mtsb_raster.spatial_ref.crs_wkt)
    data_struct["gdf"] = gdf_temp.copy()

    ds_fwi = download_and_read_fwi_data_for_dates(gdf_filtered["Ig_Date"])
    ds_fwi = ds_fwi.rio.write_crs("EPSG:4326")
    ds_fwi = ds_fwi.rename({"lat": "y", "lon": "x", fwi_variable: "fwi"})
    ds_fwi = ds_fwi.sel(x=slice(-125.4, -112.5), y=slice(32.4, 50.1))
    da_fwi = ds_fwi["fwi"].compute()

    da_fwi_reproj = da_fwi.rio.reproject_match(da_mtsb_raster)
    bounds = da_mtsb_raster.rio.bounds()
    da_fwi_reproj_clipped = da_fwi_reproj.rio.clip_box(*bounds)

    data_struct["da_fwi"] = da_fwi_reproj_clipped

    print(f"{state} {year} processed!")
    input_data.append(data_struct)


In [65]:
fire_severity_dfs = []

for data in input_data:
    da_name = data["da_fire_severity"].name
    da_temp = data["da_fire_severity"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})


    df_temp =df_mean.merge(df_max, how="inner", on="Event_ID")[["Event_ID", da_name+"_mean", da_name+"_max"]]
    fire_severity_dfs.append(df_temp.copy())

    state = data["state"]
    year = data["year"]
    print(f"{state} {year} fire severity processed!")

fwi_dfs = []

for data in input_data:
    da_name = data["da_fwi"].name
    da_temp = data["da_fwi"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})

    df_temp = pd.merge(df_max, df_mean, how="left", on=["Event_ID", "time"])[["Event_ID", "time", da_name+"_mean", da_name+"_max"]]

    fwi_dfs.append(df_temp.copy())

    state = data["state"]
    year = data["year"]
    print(f"{state} {year} FWI processed!")

fire_severity_df = pd.concat(fire_severity_dfs)
fwi_df = pd.concat(fwi_dfs)
fire_severity_df = fire_severity_df.drop_duplicates(subset=["Event_ID"])
fwi_df = fwi_df.drop_duplicates(subset=["Event_ID", "time"])

fire_df = fwi_df.merge(fire_severity_df, how='left', on="Event_ID")

mtbs_gdf_merged = mtbs_gdf.merge(fire_df, how='left', left_on=["Event_ID", "Ig_Date"], right_on=["Event_ID", "time"])


OR 2023 fire severity processed!
OR 2023 FWI processed!


In [69]:
mtbs_gdf_merged.dropna(subset="fwi_mean")

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Low_T,Mod_T,High_T,Comment,geometry,time,fwi_mean,fwi_max,fire_severity_mean,fire_severity_max
29939,OR4590811915320230613,581EB6AE-9DAA-451B-8372-42E515197461,HAT ROCK,Wildfire,10030627,MTBS,Initial,17264,45.959,-119.02,...,40,9999,9999,None,"MULTIPOLYGON (((-118.8907 45.9819, -118.89079 ...",2023-06-13,47.480586,49.885696,2.000000,2.0
29980,OR4274612434420230825,BF2E0890-DA87-47BC-9A30-0062DB83B3FE,ANVIL,Wildfire,10030745,MTBS,Extended,22261,42.752,-124.326,...,80,338,625,Used perimeter from RAVG mapping with slight a...,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",2023-08-25,5.155885,6.465652,2.404739,4.0
29989,OR4545812208520230825,8A6D09B9-4A0B-4215-9AAC-1CC17EEC19EB,CAMP CREEK,Wildfire,10030774,MTBS,Extended,2056,45.45,-122.081,...,65,346,650,Used perimeter from RAVG mapping with edits.,"POLYGON ((-122.06978 45.45618, -122.06966 45.4...",2023-08-25,14.171033,18.434149,3.055861,4.0
29993,OR4397112254620230723,A07D9467-CE0F-46B8-ACA1-209FE8A4A9B4,BEDROCK,Wildfire,10030785,MTBS,Extended,31692,43.987,-122.515,...,70,321,600,None,"POLYGON ((-122.61524 44.01428, -122.61277 44.0...",2023-07-23,24.151124,24.368780,2.619064,4.0
30017,OR4337712360720230825,D5FE3AFF-96C9-4972-AA45-87DE90DD147C,COUGAR CR. #1,Wildfire,10030845,MTBS,Extended,6971,43.367,-123.589,...,60,317,600,OR4337312358720230825 and OR433731235872023082...,"POLYGON ((-123.62554 43.39382, -123.62343 43.3...",2023-08-25,3.715164,3.974969,2.626278,4.0
30018,OR4340112273520230825,C3508196-B91F-43B1-A6BC-5EB39266B901,CHILCOOT,Wildfire,10030847,MTBS,Extended,1698,43.4,-122.725,...,60,330,625,None,"POLYGON ((-122.71406 43.40086, -122.71392 43.4...",2023-08-25,26.001451,27.226955,2.736957,4.0
30019,OR4341012348120230825,6FC52B72-B9C0-4612-850B-842423F74F1E,LIGHTHOUSE RD #3,Wildfire,10030850,MTBS,Extended,1286,43.407,-123.494,...,70,334,625,None,"POLYGON ((-123.51487 43.41063, -123.5149 43.41...",2023-08-25,3.065025,3.974969,2.335303,4.0
30027,OR4251612403720230716,76502E27-4AE0-4100-A9AD-F1850B16333D,FLAT,Wildfire,10030883,MTBS,Extended,34496,42.448,-124.081,...,50,286,550,None,"POLYGON ((-124.03739 42.51497, -124.03739 42.5...",2023-07-16,43.142180,43.688904,2.979660,4.0
30043,OR4233512140520230722,29AD6FD0-CEC2-4488-B46E-A9503B134A5C,GOLDEN,Wildfire,10030940,MTBS,Extended,2227,42.331,-121.384,...,10,252,500,None,"POLYGON ((-121.39134 42.33157, -121.39246 42.3...",2023-07-22,39.866180,39.952805,2.824684,4.0
30047,OR4292712167320230923,BED920A7-D8AE-42F6-921B-B376C2DCE30C,MARSH,Wildfire,10030961,MTBS,Initial,2684,42.933,-121.675,...,90,9999,9999,No high or moderate severity was identified du...,"POLYGON ((-121.66117 42.94817, -121.66109 42.9...",2023-09-23,27.673539,27.682623,2.000000,2.0


In [ ]:


fire_severity_df = pd.concat(fire_severity_dfs)
fire_severity_df = fire_severity_df.drop_duplicates(subset=["Event_ID"])
mtbs_gdf_merged = mtbs_gdf.merge(fire_severity_df, how='left', on="Event_ID")

OR 2023 processed!


In [55]:
mtbs_gdf.loc[mtbs_gdf["Event_ID"]=="OR4274612434420230825"]["Ig_Date"]

29980   2023-08-25
Name: Ig_Date, dtype: datetime64[ms]

In [52]:
df_temp.loc[df_temp["Event_ID"]=="OR4274612434420230825"]

,geometry_x,zonal_statistics_x,time,Event_ID,fwi_max,geometry_y,zonal_statistics_y,fwi_mean
12,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-05-20,OR4274612434420230825,6.389956,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,6.145650
13,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-06-13,OR4274612434420230825,15.274875,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,14.685420
14,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-07-06,OR4274612434420230825,27.100496,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,27.018813
15,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-07-10,OR4274612434420230825,20.434618,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,20.380749
16,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-07-16,OR4274612434420230825,33.960190,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,33.904967
17,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-07-22,OR4274612434420230825,32.162975,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,32.126916
18,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-07-23,OR4274612434420230825,33.175850,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,33.141200
19,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-08-06,OR4274612434420230825,27.812725,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,27.799849
20,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-08-25,OR4274612434420230825,6.465652,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,5.155885
21,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",max,2023-08-29,OR4274612434420230825,22.299656,"POLYGON ((-124.31229 42.7095, -124.31234 42.70...",mean,21.701645


In [51]:
mtbs_gdf.merge(df_temp, how='left', left_on=["Event_ID", "Ig_Date"], right_on=["Event_ID", "time"]).dropna(subset=["fwi_max", "fwi_mean"])[["Event_ID", "Ig_Date", "time", "fwi_max", "fwi_mean"]]

,Event_ID,Ig_Date,time,fwi_max,fwi_mean
29939,OR4590811915320230613,2023-06-13,2023-06-13,49.885696,47.480586
29980,OR4274612434420230825,2023-08-25,2023-08-25,6.465652,5.155885
29989,OR4545812208520230825,2023-08-25,2023-08-25,18.434149,14.171033
29993,OR4397112254620230723,2023-07-23,2023-07-23,24.368780,24.151124
30017,OR4337712360720230825,2023-08-25,2023-08-25,3.974969,3.715164
30018,OR4340112273520230825,2023-08-25,2023-08-25,27.226955,26.001451
30019,OR4341012348120230825,2023-08-25,2023-08-25,3.974969,3.065025
30027,OR4251612403720230716,2023-07-16,2023-07-16,43.688904,43.142180
30043,OR4233512140520230722,2023-07-22,2023-07-22,39.952805,39.866180
30047,OR4292712167320230923,2023-09-23,2023-09-23,27.682623,27.673539


In [ ]:
service_areas = gpd.read_file(eia_utility_service_areas_path)
service_areas = service_areas.to_crs(CRS)

service_areas = service_areas[["NAME", "geometry"]]
mtbs_gdf_final = mtbs_gdf_merged[["Event_ID", "Ig_Date", "BurnBndAc", "fire_severity_mean", "fire_severity_max", "fwi_mean", "fwi_max", "geometry"]]

df_final = service_areas.sjoin(mtbs_gdf_final)
df_final = df_final.drop(columns=["geometry", "index_right"])
df_final = df_final.dropna(subset="fire_severity_mean")

In [ ]:
import requests
response = requests.get("https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.GEOS-5/Default/GPM.LATE.v5/2025/FWI.GPM.LATE.v5.Daily.Default.20250102.nc", timeout=30)

In [ ]:
response

In [ ]:
with open("data/test/test.nc", 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)